In [1]:
def feature_add(df):

    ## Adding timefeatuture because these features shows important predictive power on a 25 x 20 sample grid
    df['hour'] = df['time'] / 60 // 24
    df['weekday'] = df['time'] / (24*60) // 7
    df['month'] = df['time'] / (60*24*30) // 12
    df['year'] = df['time'] / (60*24*365)
    df['day'] = df['time'] * (60*24) //365
    
    df.drop(['time'],axis = 1, inplace = True)
    print("Adding additional features by exploiting time variable")
    
    return(df)

In [2]:
def grid_cells(df):
    # No of cells in X & Y direction
    n_cell_x = 20
    n_cell_y = 40
    
    ## Calculating the pieces of X & Y
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    
    pos_x = (df.x.values / size_x).astype(int)
    pos_y = (df.y.values / size_y).astype(int)
    
    ## Calculating the grid no on 800 grid of cells
    df['grid_cell'] = pos_y * n_cell_x + pos_x
    
    print(" The grid_cell column has been added in:")
    return(df)

In [12]:
def process_grids(df_train, df_test, gid):
    
    ## Subsetting train and test on the grid_id
    
    df_train.loc[df_train.grid_cell.values == gid]
    df_test.loc[df_test.grid_cell.values == gid]
    
    row_id = df_test.row_id.values
    df_train.drop(['grid_cell','row_id'],axis = 1,inplace = True)
    df_test.drop(['grid_cell','row_id'], axis = 1, inplace = True)
    
    ## Applying random forest on this subsetted train data
    
    rfc = RandomForestClassifier(n_estimators=10)
    le = LabelEncoder()
    
    y = le.fit_transform(df_train.place_id.values)
    df_train.drop('place_id', axis = 1, inplace = True)
    
    rfc.fit(df_train,y)
    
    prediction = le.inverse_transform(np.argsort(rfc.predict_proba(df_test),axis = 1)[:,::-1][:,0:3])
    
    print(" Process of grid no: ", gid, "is complete, returning predictions to function")
    
    return(prediction,row_id)

In [10]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
import pandas as pd

def submission():
    ## Reading train & test csv files from input location
    train = pd.read_csv('train.csv/train.csv')
    test = pd.read_csv('test.csv/test.csv')
    
    preds = np.zeros((test.shape[0],3),dtype=int)
    ## Calling feature add function
    train = feature_add(train)
    test = feature_add(test)
    
    ## Calling grid cell function to make grids
    train = grid_cells(train)
    test = grid_cells(test)
    
    ## calling process_grids function to process each grid
    
    n_cell_x = 20
    n_cell_y = 40
    for grid in range((n_cell_x * n_cell_y)):
        
        prediction,row_id = process_grids(train,test,grid)
        
        preds[row_id] = prediction
        
    df_temp = pd.DataFrame(preds, columns=['l1','l2','l3'],dtype=str)
    
    submission = df_temp.l1.str.cat([df_temp.l2,df_temp.l3],sep = " ")
    
    ## Writing to csv
    print("Writing to csv file...Please wait....")
    submission.name = 'place_id'
    
    submission.to_csv('submit_23june_01.csv',index = True, header = True, index_label = 'row_id')

In [ ]:
submission() ##produce the csv file